# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
CLIENT_PORT = 62012
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", CLIENT_PORT)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Sun 2023-01-22 09:56:24.586017 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
Sun 2023-01-22 09:56:24.587045 [INFO ] - [T] Attempting to bind to 0.0.0.0:62012
Sun 2023-01-22 09:56:24.587066 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Sun 2023-01-22 09:56:24.587714 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Sun 2023-01-22 09:56:24.603746 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f7807b9e950>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7f78041f4c20>, <FilterInterface at 0x7f78041f4fb0>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }

In [17]:
await client_fiface.command_response(filter_cmd)

Sun 2023-01-22 09:58:49.038763 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:49.055073 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }


Accepted

Sun 2023-01-22 09:58:49.091881 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
LTI(client)1440 skipped 1383: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }] }
Sun 2023-01-22 09:58:49.093591 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }] }
Sun 2023-01-22 09:58:49.093642 [TRACE] - [T<-UDP] Request { 

Sun 2023-01-22 09:58:51.349075 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:51.358226 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:51.418456 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:51.420563 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:51.608393 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:53.908930 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:53.935461 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:53.988606 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:53.997912 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:54.168854 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:56.478296 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:56.482396 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:56.538315 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:56.544408 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:56.728712 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:59.028344 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:59.029444 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:59.098435 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:59.122318 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:59.278364 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

In [13]:
# XXX get endpoint programatically from client
client_endpoint = EndpointW(f"127.0.0.1:{CLIENT_PORT}")
client_endpoint

LTI(client)52 skipped 51: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
Sun 2023-01-22 09:56:29.151958 [TRACE] - [T<-F,C] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
LTI(client)53: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(client)54: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }] }
Sun 2023-01-22 09:56:29.152363 [TRACE] - [F<-T,R] Command Accepted
LTI(client)55: got transport_rsp back from Transport layer: TransportRsp::Accepted
Sun 2023-01-22 09:56:29.152468 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId 

Endpoint(127.0.0.1:62012)

Sun 2023-01-22 09:56:29.408614 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:29.427974 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:29.658733 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:29.678111 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:29.918512 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

In [14]:
#response = await server_fiface.response()
#response

Sun 2023-01-22 09:56:30.689093 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:30.702390 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:30.949029 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:30.952127 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:31.209026 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:56:35.568845 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:35.591377 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:35.828581 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:35.839842 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:36.088625 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:56:40.689001 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:40.691487 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:40.939007 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:40.943141 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:41.198371 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:56:45.808764 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:45.829050 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:46.068312 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:46.076478 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:46.319052 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:56:50.919124 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:50.936415 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:51.178544 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:51.187767 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:56:51.428870 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

...

Sun 2023-01-22 09:58:02.668677 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:02.679109 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:02.918837 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:02.928704 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:03.178436 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:07.798641 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:07.807795 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:08.048743 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:08.055911 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:08.308462 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:12.928573 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:12.936784 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:13.178807 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:13.185994 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:13.428888 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:18.018585 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:18.034887 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:18.278655 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:18.282768 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:18.538785 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

CancelledError: 

In [18]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
new_request_action = server_notifications[0]

assert new_request_action.request_action == connect_action

Sun 2023-01-22 09:59:00.038948 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:00.053240 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:00.128482 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:00.146772 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:00.298724 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:59:02.608844 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:02.630260 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:02.688726 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:02.692849 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:02.868552 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:59:05.168356 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:05.183551 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:05.228353 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:05.246631 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:05.428328 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:59:07.718982 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:07.733292 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:07.788787 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:07.795997 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:07.978721 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:59:10.279048 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:10.282147 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:10.358462 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:59:10.375816 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }


In [16]:
cookie = base64.urlsafe_b64encode(bytes(new_request_action.request_action.name.encode()))
cookie

b'RmlsdGVyTm90ZWJvb2tDbGllbnQyMzU2'

Sun 2023-01-22 09:58:26.749084 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:26.774460 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:27.008929 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:27.024622 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:27.268455 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:31.878898 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:31.884053 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:32.128918 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:32.132014 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:32.378989 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:36.978786 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:36.982909 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:37.228966 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:37.231088 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:37.488641 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:42.099044 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:42.113292 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:42.358852 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:42.362946 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:42.619011 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

Sun 2023-01-22 09:58:47.248785 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:47.275316 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:47.508349 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:47.525832 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient2356", client_version: "0.0.666" } }
Sun 2023-01-22 09:58:47.758770 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNo

In [ ]:
response_ok = ResponseCodeW("loggedin", cookie=str(cookie), server_version="0.0.666")
response_ok

In [ ]:
await server_fiface.command(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

In [ ]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

In [ ]:
new_response_code.response_code

In [ ]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()

LTI(server)11 skipped 1: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }] }
Thu 2023-01-19 23:01:47.002075 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }] }
LTI(server)12: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:47.002128 [TRACE] - [T<-UDP] Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.6

LTI(client)50: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)51: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
LTI(client)52: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:49.902378 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 

LTI(client)72: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)73: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 16, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.307333 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 16, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
LTI

LTI(client)88: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)89: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 8 }, retry_interval: 250ms }], packets: [Request { sequence: 23, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.509350 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 8 }, retry_interval: 250ms }], packets: [Request { sequence: 23, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2t

LTI(client)106: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)107: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 31, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.711448 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 31, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb

...

Thu 2023-01-19 23:01:51.317537 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 25, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317552 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 26, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317559 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317564 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 9, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 20

Thu 2023-01-19 23:01:51.519696 [TRACE] - [T<-UDP] Request { sequence: 44, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519720 [TRACE] - [T<-UDP] Request { sequence: 45, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519725 [TRACE] - [T<-UDP] Request { sequence: 46, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519730 [TRACE] - [T<-UDP] Request { sequence: 47, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
LTI(client)190: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::Endpoin

Thu 2023-01-19 23:01:51.721598 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721611 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 8, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721617 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 19, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721622 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 20, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 20

LTI(client)246: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)247: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 28 }, retry_interval: 250ms }], packets: [Request { sequence: 95, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:51.927408 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId

LTI(client)272: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-19 23:01:52.130675 [WARN ] - [F<-T,N] error sending KeepAlive during idle endpoint (Endpoint(127.0.0.1:62013)): Filter does not contain an entry for the endpoint: Endpoint(127.0.0.1:62013)
LTI(client)273: got transport notifications: []
LTI(server)338 skipped 241: received transport_cmd DropEndpoint { endpoint: Endpoint(127.0.0.1:62012) }
Thu 2023-01-19 23:01:57.040277 [INFO ] - [F<-T,N] Endpoint Endpoint(127.0.0.1:62012) timed-out. Dropping.
Thu 2023-01-19 23:01:57.040947 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:62012) }
LTI(server)339: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:57.041272 [TRACE] - [F<-T,R] Command Accepted
LTI(server)340: got transport notifications: []
